## Task1: Calculate PI in parallel with Apache Spark using the Monte Carlo Method


We have a circle of radius 0.5, enclosed by a 1 × 1 square. The area of the circle is πr2=π/4, the area of the square is 1. If we divide the area of the circle, by the area of the square we get π/4.


Total Number of points: 0
Points within circle: 0
Pi estimation:
Add points one-by-one
Animate
Speed
Reset
 Open with CodePen
One method to estimate the value of π (3.141592...) is by using a Monte Carlo method. In the demo above, we have a circle of radius 0.5, enclosed by a 1 × 1 square. The area of the circle is πr2=π/4, the area of the square is 1. If we divide the area of the circle, by the area of the square we get π/4.

We then generate a large number of uniformly distributed random points and plot them on the graph. These points can be in any position within the square i.e. between (0,0) and (1,1). If they fall within the circle, they are coloured red, otherwise they are coloured blue. We keep track of the total number of points, and the number of points that are inside the circle. If we divide the number of points within the circle, Ninner by the total number of points, Ntotal, we should get a value that is an approximation of the ratio of the areas we calculated above, π/4.

In other words,

π4 ≈ Ninner / Ntotal

π ≈ 4Ninner / Ntotal

<img src='./montecarlo.png'>

In [7]:
# number of points to be used during the simulation
num_samples = 1000
# num_samples = 100000000

In [8]:
import random

def gen_random_point(p):
    # just generate two random x and y coordinates between (0, 1), regardless the input
    x = random.random()
    y = random.random()
    return (x, y)

# point is a tuple (x, y)
def inside_the_circle(point):
    # returns true if the point is in the circle
    x = point[0]
    y = point[1]
    return (x**2 + y**2 < 1)

First let's see how we can implement the monte-carlo based Pi estimation via the standard "functional" python APIs. 

First we generate just a list which contains the same number of elements as points we need to generate randomly. With the **map** function we generate a random (x,y) coordinated for each element. With the **filer** function it is easy to get rid off all the elements which are not in the circle. The length of the remainder list can tell you, how many randomly generated points are in the circle which can be used to calculate the points_in_circle / total_points ratio.

In [13]:
num_samples = 1000000
l = range(num_samples)

n_circle = len(list(filter(inside_the_circle, map(gen_random_point, l))))

# Formula to estimate Pi
pi = 4 * ( n_circle / num_samples)
print(pi)

3.141372


Observe, as you're increasing value of of the num_samples, the error of the estimation decreases. Let's try to run the exact code with 10 * num_sample points!

In [5]:
num_samples = 10 * 100000
l = range(num_samples)
k = len(list(filter(inside_the_circle, map(gen_random_point, l))))
pi = 4 * (k / num_samples)
print(pi)

3.141528


Now it's time to write some Spark jobs. The first task is to replicate the above logic in Spark.

First you need to create a distributed data collection, which is called RDD (Resilient Distributed Dataset). We will talk about RDDs later during the class, for now, you can think about RDD as a partitioned list which can be stored in the memory of multiple node. So the available memory for your application in your computer is not a limitation anymore.

Use the **sc.parallelize(iterator)** to create an RDD first:

In [15]:
# Some boilerplate code
#import pyspark
#sc = pyspark.SparkContext()
num_samples = 10 * 1000 * 1000

# Your code comes here
rdd = sc.parallelize(range(num_samples))
rdd

PythonRDD[1] at RDD at PythonRDD.scala:53

Use the **rdd.map()**, **rdd.filter()** and **rdd.count()** methods to calculate the total number of points which fall into the circle. Use the formula above to estimate the Pi value.

In [18]:
# map (element -> element)
# filter (element -> {0, 1})
# flatMap (element -> [elemets])
# reduce (2 elements -> agg of elements)
# reduceByKey (...) <- need to perform shuffling!

rdd2 = rdd \
    .map(gen_random_point) \
    .filter(inside_the_circle) \
    .count()

pi_estimate = 4 * (points_in_circle / num_samples)
print(pi_estimate)

0.3144488
